In [10]:
# generate_fake_sales_data.py

from faker import Faker
import pandas as pd
import random
import sqlite3

fake = Faker()

# --- Define constants ---
NUM_CUSTOMERS = 500
NUM_ORDERS = 2000
NUM_SALES = 5000
NUM_PRODUCTS = 100

# --- Helper data ---
categories = ["Electronics", "Clothing", "Home", "Beauty", "Groceries"]
regions = ["North", "South", "East", "West"]
payment_methods = ["Credit Card", "Debit Card", "Cash", "Online"]

# --- Generate Products ---
products = []
for i in range(NUM_PRODUCTS):
    category = random.choice(categories)
    products.append({
        "product_id": i + 1,
        "product_name": fake.word().capitalize(),
        "category": category,
        "unit_price": round(random.uniform(10.0, 500.0), 2),
        "stock_quantity": random.randint(10, 500)
    })
products_df = pd.DataFrame(products)
# 💡 AMENDMENT 1: Reset index to drop it
products_df.reset_index(drop=True, inplace=True) 

# --- Generate Customers ---
customers = []
for i in range(NUM_CUSTOMERS):
    customers.append({
        "customer_id": i + 1,
        "name": fake.name(),
        "email": fake.email(),
        "region": random.choice(regions),
        "signup_date": fake.date_between(start_date="-2y", end_date="today")
    })
customers_df = pd.DataFrame(customers)
# 💡 AMENDMENT 2: Reset index to drop it
customers_df.reset_index(drop=True, inplace=True)

# --- Generate Orders ---
orders = []
for i in range(NUM_ORDERS):
    cust = random.choice(customers)
    orders.append({
        "order_id": i + 1,
        "customer_id": cust["customer_id"],
        "order_date": fake.date_between(start_date="-1y", end_date="today"),
        "payment_method": random.choice(payment_methods),
        "shipping_address": fake.address().replace("\n", ", ")
    })
orders_df = pd.DataFrame(orders)
# 💡 AMENDMENT 3: Reset index to drop it
orders_df.reset_index(drop=True, inplace=True)

# --- Generate Sales ---
sales = []
for i in range(NUM_SALES):
    order = random.choice(orders)
    product = random.choice(products)
    quantity = random.randint(1, 10)
    total_price = round(quantity * product["unit_price"], 2)
    sales.append({
        "sale_id": i + 1,
        "order_id": order["order_id"],
        "product_id": product["product_id"],
        "quantity": quantity,
        "unit_price": product["unit_price"],
        "total_price": total_price,
        "salesperson": fake.name(),
        "sale_date": order["order_date"]
    })
sales_df = pd.DataFrame(sales)
# 💡 AMENDMENT 4: Reset index to drop it
sales_df.reset_index(drop=True, inplace=True)

# --- Save to SQLite Database ---
conn = sqlite3.connect("sales.db")
# This line already prevents saving the index to the database:
customers_df.to_sql("customers", conn, if_exists="replace", index=False)
orders_df.to_sql("orders", conn, if_exists="replace", index=False)
products_df.to_sql("products", conn, if_exists="replace", index=False)
sales_df.to_sql("sales", conn, if_exists="replace", index=False)
conn.close()

print("✅ Fake relational sales dataset successfully generated and saved to sales.db")
print(f"Tables created: customers({len(customers_df)}), orders({len(orders_df)}), products({len(products_df)}), sales({len(sales_df)})")

✅ Fake relational sales dataset successfully generated and saved to sales.db
Tables created: customers(500), orders(2000), products(100), sales(5000)


In [11]:
import sqlite3
conn = sqlite3.connect("sales.db")
print(conn.execute("SELECT * FROM sales").fetchone())
conn.close()

(1, 733, 27, 10, 60.47, 604.7, 'Lisa Simmons', '2024-12-22')
